# NLP (Natural Language Processing) Kullanarak IMDB Film Yorumlarını Duygu Analizi (Sentiment Analysis)

*yorumlar kaggle üzerinden dataset olarak alınmıştır.

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [32]:
#dataseti yüklüyoruz

df = pd.read_csv('NLPlabeledData.tsv', delimiter="\t", quoting=3)

#delimiter parametresi ile verinin nasıl ayrıldığını belirtmeniz gerekmektedir. Bu parametre \t olarak ayarlanmışsa, bu da verinin tab karakteri ile ayrıldığını gösterir.

#quoting parametresi genellikle CSV dosyalarında, verinin içinde çift tırnak kullanımını düzenlemek için kullanılır.

In [33]:
df.head()

id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...

In [34]:
len(df)

25000

In [35]:
len(df["review"])

25000

Durdurma kelimeleri `(stop words)`, dil işleme uygulamalarında genellikle önemsiz kabul edilen ve metinlerde sıkça kullanılan kelimelerdir. Bu kelimeler genellikle cümlenin yapısı veya anlamı için önemli değildir ve analiz veya işleme sırasında çıkarılabilir. Örnek olarak "ve", "ama", "veya", "bir", "ben", "sen", "onlar" gibi kelimeler durdurma kelimelerine örnektir.

In [36]:
nltk.download('stopwords')

#Bu komutu çalıştırdığınızda, NLTK durdurma kelimeleri veri kaynağını indirip kullanıma hazır hale getirecektir.

[nltk_data] Downloading package stopwords to /home/gizem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### veri temizleme işlemi

`BeautifulSoup`, web sayfalarındaki HTML ve XML verilerini işlemek ve analiz etmek için kullanılan bir Python kütüphanesidir. Bu kütüphane, web scraping (veri çekme) işlemlerinde sıkça kullanılan bir araçtır. BeautifulSoup, karmaşık HTML veya XML belgelerini ayrıştırmak, içeriklerini çıkarmak ve bu içerikleri manipüle etmek için kullanılır.

BeautifulSoup'un temel görevleri şunlar arasında yer alır:

HTML ve XML Ayrıştırma: BeautifulSoup, web sayfalarının HTML veya XML kodlarını ayrıştırarak daha kolay işlenebilir bir yapıya dönüştürür. Bu sayede sayfa içeriğine daha rahat erişebilirsiniz.

Veri Çıkarma: Ayrıştırılmış belgeler üzerinde gezinerek istediğiniz verileri (örneğin başlıklar, metinler, resimler, bağlantılar vb.) çıkarabilirsiniz.

Belge Manipülasyonu: BeautifulSoup, ayrıştırılmış belgelerin içeriğini düzenlemek, ekleme yapmak veya silmek gibi manipülasyonları da kolayca gerçekleştirmenizi sağlar.

CSS ve DOM Benzeri İşlemler: BeautifulSoup, web sayfalarının yapısına benzer bir şekilde elemanlara (tag, class, id) CSS seçicileri gibi yöntemlerle erişmenizi sağlar. Bu da belge üzerinde daha spesifik işlemler yapmanıza yardımcı olur.

Veri Temizliği: Web scraping işlemleri sırasında alınan veriler genellikle gereksiz veya istenmeyen karakterler içerebilir. BeautifulSoup, bu verileri temizlemenizi sağlar.

örnek tek bir review silerek işlemin nasıl yapıldığını anlayalım:

In [37]:
sample_review = df.review[0]
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [38]:
# HTML taglerini temizliyoruz
sample_review = BeautifulSoup(sample_review).get_text()
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [39]:
# regex kullanarak noktalama işaretleri ve sayıları temizliyoruz
sample_review = re.sub("[^a-zA-Z]", ' ', sample_review)  #harf olmayan karakterleri boşlukla (' ') değiştirir. 
sample_review

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [40]:
# ml algoritmamızın doğru çalışması için tüm harfleri küçük yapıyoruz

sample_review = sample_review.lower()
sample_review

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [41]:
#split metodu ile kelimeleri bölüyoruz 
sample_review = sample_review.split()
sample_review

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

In [42]:
len(sample_review)

437

In [43]:
#böldüğümüz kelimelerden stopwords kelimeleri çıkartıyoruz

swords = set(stopwords.words("english"))  # İngilizce dilindeki durdurma kelimeleri kümesini alıyoruz.
sample_review = [w for w in sample_review if w not in swords]  #sadece durdurma kelimeleri olmayan kelimeleri yeni bir liste olarak alıyoruz
sample_review

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [44]:
len(sample_review) #stopwords çıkarılmış kelime uzunluğuna bakalım

219

fonksiyon oluşturarak tüm dataframe içinde bulunan reviewları döngü içinde topluca temizliyoruz

In [45]:
def process(review):
    #HTML tag çıkarılmış hali:
    review = BeautifulSoup(review).get_text()

    #harf olmayan karakterleri boşlukla değiştirdiğimiz hali:
    review = re.sub("[^a-zA-Z]", ' ', review)

    #küçük harfe çeviriyoruz ve kelimeleri ayırıyoruz:
    review = review.lower()
    review = review.split()

    #stopwords çıkarılmış hali:
    swords = set(stopwords.words("english"))
    review = [w for w in review if w not in swords]

    #ayrılmış paragrafları geri birleştiriyoruz
    return(" ".join(review))



In [46]:
#training datamızı yukarıdaki fonksiyon yardımı ile temizliyoruz

train_x_tum = []
for r in range(len(df["review"])):
    if (r+1)%1000 == 0: #her 1000 review sonrası bir satır yazdırarak durumuna bakıyoruz
        print("No of reviews processed = ", r+1)
    train_x_tum.append(process(df["review"][r]))

/home/gizem/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


No of reviews processed =  1000
No of reviews processed =  2000
No of reviews processed =  3000
No of reviews processed =  4000
No of reviews processed =  5000
No of reviews processed =  6000
No of reviews processed =  7000
No of reviews processed =  8000
No of reviews processed =  9000
No of reviews processed =  10000
No of reviews processed =  11000
No of reviews processed =  12000
No of reviews processed =  13000
No of reviews processed =  14000
No of reviews processed =  15000
No of reviews processed =  16000
No of reviews processed =  17000
No of reviews processed =  18000
No of reviews processed =  19000
No of reviews processed =  20000
No of reviews processed =  21000
No of reviews processed =  22000
No of reviews processed =  23000
No of reviews processed =  24000
No of reviews processed =  25000


In [47]:
#train ve test split

X = train_x_tum
y = np.array(df["sentiment"])

train_X, test_X, y_train, y_test = train_test_split(X,y, test_size=0.1)

## Bag of Words
Bag of Words (BoW), Doğal Dil İşleme (NLP) alanında sıkça kullanılan bir metin temsili yöntemidir. Bu yöntem, metinleri sayısal verilere dönüştürmek ve metinleri analiz etmek için kullanılan bir yaklaşımdır. Temelde, metindeki her kelimenin frekansına bakarak bir kelime vektörü oluşturur.

BoW yöntemi şu adımları içerir:

Kelimeleri Ayıklama: Öncelikle metni kelimelere ayırarak işleme başlanır. Bu adımda noktalama işaretleri ve gereksiz karakterler temizlenir.

Kelime Sözlüğünü Oluşturma: Tüm belgedeki farklı kelimeler bir sözlük oluşturacak şekilde indekslenir. Bu sözlük, her bir kelimenin bir benzersiz sayısal bir indeksiyle eşleştirildiği bir yapıdır.

Kelime Frekanslarını Hesaplama: Her kelimenin belgedeki frekansı (kaç kez geçtiği) hesaplanır. Bu, metindeki her kelimenin belge içerisindeki önemini gösterir.

Kelime Vektörlerini Oluşturma: Kelime frekanslarına göre, her kelimenin sözlükteki indeksi temel alınarak bir vektör oluşturulur. Her bir boyut, bir kelimenin belgedeki frekansını temsil eder.

Metin Temsilini Oluşturma: Oluşturulan kelime vektörleri bir araya getirilerek metnin sayısal bir temsili elde edilir.



In [48]:
#sklearn içinde bulunan countvectorizer fonksiyonunu kullanarak max 5000 kelimelik bag of words oluşturuyoruz
vectorizer = CountVectorizer(max_features=5000)

#train verilerimizi feature vektör matrisine çeviriyoruz
train_X = vectorizer.fit_transform(train_X)

In [49]:
train_X

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1777522 stored elements in Compressed Sparse Row format>

In [50]:
#fit işlemi için arraye dönüştürüyoruz

train_X = train_X.toarray()
train_y = y_train

In [51]:
train_X.shape, train_y.shape

((22500, 5000), (22500,))

In [52]:
train_y

array([0, 1, 1, ..., 1, 0, 0])

### Random Forest Modeli oluşturup fit ediyoruz


In [53]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_X, train_y)

RandomForestClassifier(random_state=42)

test verilerimiz için bag of words işlemini tekrarlıyoruz

In [54]:
test_XX = vectorizer.transform(test_X)

In [55]:
test_XX

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 197362 stored elements in Compressed Sparse Row format>

In [56]:
test_XX = test_XX.toarray()

In [57]:
test_XX.shape

(2500, 5000)

## prediction

In [58]:
test_predict = model.predict(test_XX)
dogruluk = roc_auc_score(y_test, test_predict)

In [59]:
print("Dogruluk Orani : % ", dogruluk*100)

Dogruluk Orani : %  85.99597683383351
